In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import pandas as pd
import datetime

dase_df=pd.read_csv('/content/drive/MyDrive/signate/シェアサイクルの利用予測/dase_df.csv',parse_dates=["date","installation_date"])
dase_df.head()

,id,year,month,day,hour,station_id,bikes_available,predict,date,rent_num,...,weather1,weather2,lat,long,dock_count,city,installation_date,is_install,diff,weekday
0,0.0,2013.0,9.0,1.0,0.0,0.0,11.0,0.0,2013-09-01 00:00:00,0.0,...,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True,0.0,6
1,1.0,2013.0,9.0,1.0,1.0,0.0,11.0,0.0,2013-09-01 01:00:00,0.0,...,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True,0.0,6
2,2.0,2013.0,9.0,1.0,2.0,0.0,11.0,0.0,2013-09-01 02:00:00,0.0,...,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True,0.0,6
3,3.0,2013.0,9.0,1.0,3.0,0.0,11.0,0.0,2013-09-01 03:00:00,0.0,...,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True,0.0,6
4,4.0,2013.0,9.0,1.0,4.0,0.0,11.0,0.0,2013-09-01 04:00:00,0.0,...,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True,0.0,6


In [5]:
station_num=70
dfs=[0]*station_num
for i in range(station_num):
    dfs[i]=[dase_df.loc[(dase_df["station_id"]==i)&(dase_df["is_install"]==True)&(dase_df["hour"]==j),:].reset_index(drop=True) for j in range(24)]
    
    

In [ ]:
from scipy import stats
import numpy as np

each_mean=[[0]*24]*station_num
each_std=[[0]*24]*station_num
data=[[0]*24]*station_num

for i in range(station_num):
    for j in range(24):
        tmp=dfs[i][j]["diff"].to_numpy(dtype=float)
        each_mean=tmp.mean(axis=0)
        each_std=np.std(tmp, axis=0)
        data[i][j]=stats.zscore(tmp)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

def get_data(dfs,station,hour):
    margin=4*7
    tmp=dfs[station][hour]
    X=[]
    Y=[]
    for i in range(len(tmp)-margin):
        rows=tmp.loc[i:i+margin,:]
        if rows.iloc[-1]["predict"]==0:
            x=rows.iloc[0:-1][["bikes_available","weather1","weather2","diff","weekday"]].values
            #x=rows.iloc[0:-1]["bikes_available"].values
            y=rows.iloc[-1]["diff"]
            X.append(x)
            Y.append(y)
    X=torch.FloatTensor(X)
    Y=torch.FloatTensor(Y)
    train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size = 0.1)
    #return (train_x,test_x,train_y,test_y)
    train_dataset=TensorDataset(train_x,train_y)
    train_loader=DataLoader(train_dataset)
    test_dataset=TensorDataset(test_x,test_y)
    test_loader=DataLoader(test_dataset)
    return (train_loader,test_loader)

def get_submit_data(dfs,station,hour):
    margin=4*7
    tmp=dfs[station][hour]
    id=[]
    X=[]
    for i in range(len(tmp)-margin):
        rows=tmp.loc[i:i+margin,:]
        if i%10000==0:
            print(i)
        if rows.iloc[-1]["predict"]==1:
            id.append(rows.iloc[-1]["id"])
            X.append(rows.iloc[0:-1][["bikes_available","weather1","weather2","diff","weekday"]].values)
            #X.append(rows.iloc[0:-1]["bikes_available"].values)
            #Y.append(rows.iloc[-1]["diff"])
    return (id,X)

import torch.nn as nn
from torch import optim
import torch.nn.functional as F

def train_model(train_x,test_x,train_y,test_y):
    lgb_train = lgb.Dataset(train_x, train_y)
    lgb_test = lgb.Dataset(test_x, test_y, reference=lgb_train)

    # LightGBM のハイパーパラメータ
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
    }
    # 上記のパラメータでモデルを学習する
    model = lgb.train(params,
                    train_set=lgb_train, # トレーニングデータの指定
                    valid_sets=lgb_test, # 検証データの指定
                    )
    return model
# テストデータを予測する
#pred_y = model.predict(test_x, num_iteration=model.best_iteration)

In [ ]:
models=[[0]*24]*station_num
for i in range(station_num):
    for j in range(24):
        train_x,test_x,train_y,test_y=get_data(dfs,i,j)
        models[i][j]=train_model(train_x,test_x,train_y,test_y)

id_lst=[]
diff_lst=[]
for i in range(station_num):
    for j in range(24):
        id,X=get_submit_data(dfs,i,j)
        id_lst.extend(id)
        if len(X)>0:
            print(id)
            diff_lst.extend(models[i][j].predict(X, num_iteration=models[i][j].best_iteration))

for i in range(len(id_lst)):
    if i%10000==0:
        print(i)
    marged_df.loc[id_lst[i],"pred_bikes_available"]=diff_lst[i]